<a href="https://colab.research.google.com/github/PedroMurta/Desafio-Dashboard/blob/main/teste_dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [91]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.float_format = '{:.2f}'.format

In [92]:
# Tabela Policies
df_policies = pd.read_excel('fake_allocation_policies.xlsx', names=['class_name', 'conservador', 'moderado-conservador', 'moderado', 'moderado-agressivo', 'agressivo'])

# Tabela Position
df_position = pd.read_csv('fake_position.csv')


# Tabela account 
accounts = df_position[['account_code', 'account_suitability']]


# Tabela asset 
asset = df_position['asset_name'].drop_duplicates()
asset = pd.DataFrame(asset)
asset.insert(0, 'id',  range(0, 0 + len(asset)))
asset.head()

account = accounts.drop_duplicates()

In [93]:
df_position.head()

,account_code,account_suitability,asset_name,asset_cnpj,class_name,position_value
0,65547.00,moderado,CDB PRE DU CDB120CAG00,NaN,Renda Fixa Pré-Fixada,133886.22
1,65547.00,moderado,CDB PRE DU CDB320A2QHT,NaN,Renda Fixa Pré-Fixada,61127.17
2,65547.00,moderado,CDB PRE DU CDB420IBE9D,NaN,Renda Fixa Pré-Fixada,206855.53
3,65547.00,moderado,CDB PRE DU CDB321ALNBI,NaN,Renda Fixa Pré-Fixada,47910.69
4,65547.00,moderado,LCI PRE 21K00148900,NaN,Renda Fixa Pré-Fixada,71442.63


In [94]:
position = df_position.copy()
policies = df_policies.copy()

position['account_code'].fillna(0, inplace=True)
position['account_code'] = position['account_code'].astype(int)

#position['position_value'] = position['position_value'].astype(float)

In [95]:
df_agrupado_classe = position.groupby(['account_code', 'account_suitability','class_name']).agg({'position_value': 'sum'}).reset_index()
df_agrupado_classe.head(9)

,account_code,account_suitability,class_name,position_value
0,2429,moderado,Renda Fixa Inflação,134310122.39
1,2429,moderado,Renda Fixa Pré-Fixada,296872.54
2,2429,moderado,Renda Fixa Pós-Fixada,6425.97
3,2429,moderado,Reserva de Liquidez,13061.73
4,2429,moderado,Saldo em Conta,0.00
5,6777,moderado-conservador,Alternativos,32197.54
6,6777,moderado-conservador,Multimercado,20820.84
7,6777,moderado-conservador,Renda Fixa Inflação,32335.25
8,6777,moderado-conservador,Renda Fixa Pré-Fixada,18299.89


In [96]:
df_agrupado_suitability = df_agrupado_classe.groupby(['account_code', 'account_suitability']).agg({'position_value': 'sum'}).reset_index()
df_agrupado_suitability

,account_code,account_suitability,position_value
0,2429,moderado,134626482.63
1,6777,moderado-conservador,470566.94
2,7826,moderado-conservador,183962.86
3,18073,moderado-conservador,65559838.14
4,20921,agressivo,27915.92
5,32263,moderado-conservador,278274189.45
6,32449,moderado-conservador,1726743.04
7,45848,moderado,125103.31
8,46168,moderado,162024.01
9,46744,moderado-agressivo,37913.63


In [97]:
df_agrupado_suitability['total_value'] = df_agrupado_suitability['position_value']

In [98]:
total_agrupado = pd.merge(df_agrupado_suitability, df_agrupado_classe, on='account_code', how='left')
totao_agrupado = total_agrupado[['account_code', 'total_value']].drop_duplicates()

In [99]:
total_agrupado.value_counts().sum()

24

In [100]:
position['account_suitability']

0                  moderado
1                  moderado
2                  moderado
3                  moderado
4                  moderado
               ...         
529             conservador
530      moderado-agressivo
531               agressivo
532    moderado-conservador
533                     NaN
Name: account_suitability, Length: 534, dtype: object

In [101]:
def get_metrica(row):
    politica = row['account_suitability']
    if politica == 'conservador':
        return row['conservador']
    elif politica == 'moderado conservador':
        return row['moderado_conservador']
    elif politica == 'moderado':
        return row['moderado']
    elif politica == 'moderado agressivo':
        return row['moderado_agressivo']
    elif politica == 'agressivo':
        return row['agressivo']
    else:
        return None


merged = pd.merge(position, policies, on='class_name')

merged['metrica'] = merged.apply(get_metrica, axis=1)

merged.head()

,account_code,account_suitability,asset_name,asset_cnpj,class_name,position_value,conservador,moderado-conservador,moderado,moderado-agressivo,agressivo,metrica
0,65547,moderado,CDB PRE DU CDB120CAG00,NaN,Renda Fixa Pré-Fixada,133886.22,0.05,0.08,0.10,0.11,0.10,0.10
1,65547,moderado,CDB PRE DU CDB320A2QHT,NaN,Renda Fixa Pré-Fixada,61127.17,0.05,0.08,0.10,0.11,0.10,0.10
2,65547,moderado,CDB PRE DU CDB420IBE9D,NaN,Renda Fixa Pré-Fixada,206855.53,0.05,0.08,0.10,0.11,0.10,0.10
3,65547,moderado,CDB PRE DU CDB321ALNBI,NaN,Renda Fixa Pré-Fixada,47910.69,0.05,0.08,0.10,0.11,0.10,0.10
4,65547,moderado,LCI PRE 21K00148900,NaN,Renda Fixa Pré-Fixada,71442.63,0.05,0.08,0.10,0.11,0.10,0.10


In [102]:
df_agrupado_classe = merged.groupby(['account_code', 'account_suitability','class_name', 'metrica']).agg({'position_value': 'sum'}).reset_index()
df_agrupado_classe.head(9)

,account_code,account_suitability,class_name,metrica,position_value
0,2429,moderado,Renda Fixa Inflação,0.24,134310122.39
1,2429,moderado,Renda Fixa Pré-Fixada,0.10,296872.54
2,2429,moderado,Renda Fixa Pós-Fixada,0.30,6425.97
3,2429,moderado,Saldo em Conta,0.00,0.00
4,20921,agressivo,Renda Fixa Pós-Fixada,0.05,20237.32
5,20921,agressivo,Renda Variável,0.20,4251.29
6,20921,agressivo,Saldo em Conta,0.00,3.65
7,45848,moderado,Multimercado,0.22,26015.53
8,45848,moderado,Renda Fixa Inflação,0.24,5604.31


In [103]:
df = pd.merge(df_agrupado_classe, total_agrupado, on='account_code')
df.head()

,account_code,account_suitability,class_name,metrica,position_value,total_value
0,2429,moderado,Renda Fixa Inflação,0.24,134310122.39,134626482.63
1,2429,moderado,Renda Fixa Pré-Fixada,0.10,296872.54,134626482.63
2,2429,moderado,Renda Fixa Pós-Fixada,0.30,6425.97,134626482.63
3,2429,moderado,Saldo em Conta,0.00,0.00,134626482.63
4,20921,agressivo,Renda Fixa Pós-Fixada,0.05,20237.32,27915.92


In [104]:
round(134626482.63 * 0.24,1)

32310355.8

In [105]:
df['valor_atual'] = df['position_value']
df.head(7)

,account_code,account_suitability,class_name,metrica,position_value,total_value,valor_atual
0,2429,moderado,Renda Fixa Inflação,0.24,134310122.39,134626482.63,134310122.39
1,2429,moderado,Renda Fixa Pré-Fixada,0.10,296872.54,134626482.63,296872.54
2,2429,moderado,Renda Fixa Pós-Fixada,0.30,6425.97,134626482.63,6425.97
3,2429,moderado,Saldo em Conta,0.00,0.00,134626482.63,0.00
4,20921,agressivo,Renda Fixa Pós-Fixada,0.05,20237.32,27915.92,20237.32
5,20921,agressivo,Renda Variável,0.20,4251.29,27915.92,4251.29
6,20921,agressivo,Saldo em Conta,0.00,3.65,27915.92,3.65


In [106]:
valor = 134626482.63
v = 0.24
valor_formatado = '{:,.2f}'.format(valor * v)

valor_formatado

print(valor_formatado)
(134310122.39 > 134626482.63 * 0.24)

32,310,355.83


True

In [107]:
p = (134310122.39 / 134626482.63) * 100
round(p, 2)

99.77

In [108]:
df['valor_esperado'] = round(df['total_value'] * df['metrica'], 3)
df.head(7)

,account_code,account_suitability,class_name,metrica,position_value,total_value,valor_atual,valor_esperado
0,2429,moderado,Renda Fixa Inflação,0.24,134310122.39,134626482.63,134310122.39,32310355.83
1,2429,moderado,Renda Fixa Pré-Fixada,0.10,296872.54,134626482.63,296872.54,13462648.26
2,2429,moderado,Renda Fixa Pós-Fixada,0.30,6425.97,134626482.63,6425.97,40387944.79
3,2429,moderado,Saldo em Conta,0.00,0.00,134626482.63,0.00,0.00
4,20921,agressivo,Renda Fixa Pós-Fixada,0.05,20237.32,27915.92,20237.32,1395.80
5,20921,agressivo,Renda Variável,0.20,4251.29,27915.92,4251.29,5583.18
6,20921,agressivo,Saldo em Conta,0.00,3.65,27915.92,3.65,0.00


In [109]:
df['valor_porcentagem_atual'] = round(df['position_value'] / df['total_value'] * 100, 3)
df.head(13)

,account_code,account_suitability,class_name,metrica,position_value,total_value,valor_atual,valor_esperado,valor_porcentagem_atual
0,2429,moderado,Renda Fixa Inflação,0.24,134310122.39,134626482.63,134310122.39,32310355.83,99.77
1,2429,moderado,Renda Fixa Pré-Fixada,0.10,296872.54,134626482.63,296872.54,13462648.26,0.22
2,2429,moderado,Renda Fixa Pós-Fixada,0.30,6425.97,134626482.63,6425.97,40387944.79,0.01
3,2429,moderado,Saldo em Conta,0.00,0.00,134626482.63,0.00,0.00,0.00
4,20921,agressivo,Renda Fixa Pós-Fixada,0.05,20237.32,27915.92,20237.32,1395.80,72.49
5,20921,agressivo,Renda Variável,0.20,4251.29,27915.92,4251.29,5583.18,15.23
6,20921,agressivo,Saldo em Conta,0.00,3.65,27915.92,3.65,0.00,0.01
7,45848,moderado,Multimercado,0.22,26015.53,125103.31,26015.53,27522.73,20.80
8,45848,moderado,Renda Fixa Inflação,0.24,5604.31,125103.31,5604.31,30024.79,4.48
9,45848,moderado,Renda Fixa Pré-Fixada,0.10,4161.47,125103.31,4161.47,12510.33,3.33


In [110]:
# Elevar a 0.5 é a mesma coisa que tirar a raiz quadrada
df['distancia'] = ((df['valor_atual'] - df['valor_esperado']) ** 2)**0.5
df.head(7)

,account_code,account_suitability,class_name,metrica,position_value,total_value,valor_atual,valor_esperado,valor_porcentagem_atual,distancia
0,2429,moderado,Renda Fixa Inflação,0.24,134310122.39,134626482.63,134310122.39,32310355.83,99.77,101999766.56
1,2429,moderado,Renda Fixa Pré-Fixada,0.10,296872.54,134626482.63,296872.54,13462648.26,0.22,13165775.72
2,2429,moderado,Renda Fixa Pós-Fixada,0.30,6425.97,134626482.63,6425.97,40387944.79,0.01,40381518.82
3,2429,moderado,Saldo em Conta,0.00,0.00,134626482.63,0.00,0.00,0.00,0.00
4,20921,agressivo,Renda Fixa Pós-Fixada,0.05,20237.32,27915.92,20237.32,1395.80,72.49,18841.52
5,20921,agressivo,Renda Variável,0.20,4251.29,27915.92,4251.29,5583.18,15.23,1331.89
6,20921,agressivo,Saldo em Conta,0.00,3.65,27915.92,3.65,0.00,0.01,3.65


In [111]:
df['aderencia'] = 1 - (1 + df['distancia'])
df.head(7)

,account_code,account_suitability,class_name,metrica,position_value,total_value,valor_atual,valor_esperado,valor_porcentagem_atual,distancia,aderencia
0,2429,moderado,Renda Fixa Inflação,0.24,134310122.39,134626482.63,134310122.39,32310355.83,99.77,101999766.56,0.00
1,2429,moderado,Renda Fixa Pré-Fixada,0.10,296872.54,134626482.63,296872.54,13462648.26,0.22,13165775.72,0.00
2,2429,moderado,Renda Fixa Pós-Fixada,0.30,6425.97,134626482.63,6425.97,40387944.79,0.01,40381518.82,0.00
3,2429,moderado,Saldo em Conta,0.00,0.00,134626482.63,0.00,0.00,0.00,0.00,1.00
4,20921,agressivo,Renda Fixa Pós-Fixada,0.05,20237.32,27915.92,20237.32,1395.80,72.49,18841.52,0.00
5,20921,agressivo,Renda Variável,0.20,4251.29,27915.92,4251.29,5583.18,15.23,1331.89,0.00
6,20921,agressivo,Saldo em Conta,0.00,3.65,27915.92,3.65,0.00,0.01,3.65,0.22
